## Algoritmos Genéticos com a biblioteca DEAP (Distributed Evolutionary Algorithms in Python)

[Github da biblioteca Deap](https://github.com/deap/deap)

Nesta biblioteca há vários outros algoritmos de otimização, no link do github acima há um readme que apresenta todos os algoritmos que são possíveis de utilizar.

Iremos utilizar do mesmo contexto do arquivo *algoritmo_genetico.ipynb*, porém agora utilizando a biblioteca Deap.

In [1]:
import random
import numpy
from deap import base
from deap import creator
from deap import algorithms
from deap import tools
import matplotlib.pyplot as plt

### Criando a classe produtos

In [2]:
class Produto():
    def __init__(self, nome, espaco, valor):
        self.nome = nome
        self.espaco = espaco
        self.valor = valor

In [3]:
lista_produtos: list[Produto] = []

lista_produtos.append(Produto("Geladeira Dako", 0.751, 999.90))
lista_produtos.append(Produto("Iphone 6", 0.0000899, 2911.12))
lista_produtos.append(Produto("TV 55' ", 0.400, 4346.99))
lista_produtos.append(Produto("TV 50' ", 0.290, 3999.90))
lista_produtos.append(Produto("TV 42' ", 0.200, 2999.00))
lista_produtos.append(Produto("Notebook Dell", 0.00350, 2499.90))
lista_produtos.append(Produto("Ventilador Panasonic", 0.496, 199.90))
lista_produtos.append(Produto("Microondas Electrolux", 0.0424, 308.66))
lista_produtos.append(Produto("Microondas LG", 0.0544, 429.90))
lista_produtos.append(Produto("Microondas Panasonic", 0.0319, 299.29))
lista_produtos.append(Produto("Geladeira Brastemp", 0.635, 849.00))
lista_produtos.append(Produto("Geladeira Consul", 0.870, 1199.89))
lista_produtos.append(Produto("Notebook Lenovo", 0.498, 1999.90))
lista_produtos.append(Produto("Notebook Asus", 0.527, 3999.00))

In [4]:
espacos = []
valores = []
nomes = []

limite = 3

for p in lista_produtos:
    espacos.append(p.espaco)
    valores.append(p.valor)
    nomes.append(p.nome)

### Inicializando a biblioteca

**Inicialização dos recursos da biblioteca**

In [5]:
toolbox = base.Toolbox()

A função FitnessMax irá realizar a maximização, no nosso caso, do valor dos produtos.

O termo se refere ao processo de avaliação.

weights=(1.0,) -> Pesos de como a função fitness irá trabalhar. 

Em outras palavras, aqui não teremos o valor em reais (somatório dos valores dos produtos), mas sim um valor entre 0 e 1, onde quanto mais próximo de 1 (um) melhor é a nota.

In [6]:
# Definindo a função de avaliação
creator.create('FitnessMax', base.Fitness, weights=(1.0,))

**Criando o indivíduo**

In [7]:
creator.create('Individual',
               list, # Os cromossomos do indivíduo ficarão armazenados aqui
               fitness=creator.FitnessMax)

Registrando como o atributo list do *Individual* será preenchido.

In [8]:
toolbox.register('attr_bool', # Atributo booleano
                 random.randint, # Número randomico
                 0, 1) # Entre 0 e 1 (vai levar ou não vai levar o produto)

Registrando a classe Individual (indivíduo).

In [9]:
toolbox.register('individual',
                 tools.initRepeat,
                 creator.Individual,
                 toolbox.attr_bool,
                 n=len(espacos))

Função para fazer a criação da população.

In [10]:
toolbox.register('population',
                 tools.initRepeat,
                 list, # Tipo de dados que vão estar as soluções
                 toolbox.individual)

### Função de avaliação

É necessário criar a função de avaliação pois a biblioteca não consegue "adivinhar" como realizar a avaliação dos indivíduos.

É necessário realizar a divisão no retorno da nota para normalizar os resultados entre 0 e 1.

In [11]:
def avaliacao(individual):
    nota = 0
    soma_espacos = 0
    
    for i in range(len(individual)): # Tamanho do indivíduo, nada mais é do que o cromossomo
        if individual[i] == 1:
            nota += valores[i] # Soma os valores de cada produto
            soma_espacos += espacos[i] # Soma os espaços ocupados por cada produto
    
    if soma_espacos > limite:
        nota = 1
        
    return nota / 100000, # Essa vírgula no final é necessária, pois definimos os pesos como weights=(1.0,)

**Registrando a função de avaliação**

In [12]:
toolbox.register('evaluate', avaliacao)

**Registrando como será feito o cruzamento.**

cvOnePoint: 1 ponto de corte (semelhante à implementação manual do arquivo *algoritmo_genetico.ipynb*).

In [13]:
toolbox.register('mate', tools.cxOnePoint) # cx significa crossover

**Registrando como será feita a mutação.**

mutFlipBit: Mudança de bits, quando é 0 se transforma em 1 e vice versa.

In [14]:
toolbox.register('mutate',
                 tools.mutFlipBit,
                 indpb=0.01) # Taxa/probabilidade de mutação

**Registrando como será feita a seleção dos indivíduos para o cruzamento e mutação.**

Utilizando o método da roleta viciada, assim como na implementação manual.

In [15]:
toolbox.register('select', tools.selRoulette) # Roleta viciada

### Função principal

Criando a população.

É possível acessar a classe criada anteriormente (com o toolbox.register) referenciando-a.

**Seed para ter reprodutibilidade dos resultados**

In [16]:
random.seed(487)

In [17]:
populacao = toolbox.population(n=20) # População de 20 indivíduos
probabilidade_crossover = 1.0 # Probabilidade de 100% para crossover (cruzamento), irá sobrescrever toda a população após o crossover
probabilidade_mutacao = 0.01
numero_geracoes = 100

# Irá utilizar os valores da função Fitness (função de avaliação)
estatisticas = tools.Statistics(key=lambda individuo: individuo.fitness.values)

# Definindo a função que pegará o valor máximo, mínimo, média e desvio padrão
# Registrando esses valores, será mostrado no momento que o algoritmo rodar
estatisticas.register('max', numpy.max)
estatisticas.register('min', numpy.min)
estatisticas.register('mean', numpy.mean)
estatisticas.register('std', numpy.std)

Tipo de algoritmo utilizado: eaSimple (Algoritmo genético simples).

In [18]:
populacao, info = algorithms.eaSimple(population=populacao,
                                      toolbox=toolbox,
                                      cxpb=probabilidade_crossover,
                                      mutpb=probabilidade_mutacao,
                                      ngen=numero_geracoes,
                                      stats=estatisticas)

gen	nevals	max     	min  	mean    	std     
0  	20    	0.171439	1e-05	0.078515	0.066156
1  	20    	0.206465	0.0545595	0.125558	0.0406552
2  	20    	0.196466	1e-05    	0.114101	0.0407693
3  	20    	0.171087	0.0680675	0.122972	0.0315358
4  	20    	0.199937	0.0680675	0.128471	0.033294 
5  	20    	0.199937	0.0693736	0.145034	0.0333342
6  	20    	0.199937	0.107356 	0.165701	0.0270294
7  	20    	0.199937	0.107356 	0.162676	0.0219319
8  	20    	0.199937	0.127355 	0.171673	0.0216599
9  	20    	0.214937	0.151468 	0.183141	0.0179121
10 	20    	0.199937	0.137365 	0.183924	0.0169265
11 	20    	0.199937	0.167354 	0.189249	0.012304 
12 	20    	0.199937	0.171467 	0.192854	0.00952553
13 	20    	0.199937	0.171467 	0.190528	0.0110879 
14 	20    	0.239936	0.156467 	0.188375	0.0179343 
15 	20    	0.239936	0.156467 	0.190875	0.0171273 
16 	20    	0.239936	0.156467 	0.198048	0.0178219 
17 	20    	0.239936	0.171467 	0.198604	0.011197  
18 	20    	0.199937	0.196466 	0.197507	0.00159057
19 	20    	0.199937	0.1